In [ ]:
#!pip install keras-tuner

In [1]:
import pandas as pd
import matplotlib as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
stardf = pd.read_csv("..//TG_stars.csv")
stardf.head (7)

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass
0,9.10,3.54,1.39,0.482,16.845016,3
1,9.27,21.90,3.10,0.999,20.972221,5
2,6.61,2.81,0.63,-0.019,13.853532,1
3,8.06,7.75,0.97,0.370,17.506509,3
4,8.55,2.87,1.11,0.902,15.839409,4
5,12.31,18.80,4.99,1.336,23.680789,6
6,9.64,17.74,1.30,0.740,20.884768,4


In [3]:
stars_df_features = stardf.copy()

stars_df_features['Amag_SQ'] = stars_df_features['Amag']**2
stars_df_features['Vmag_SQ'] = stars_df_features['Vmag']**2
stars_df_features['B-V_SQ'] = stars_df_features['B-V']**2
stars_df_features['Sum_AV'] = stars_df_features['Amag'] + stars_df_features['Vmag']
stars_df_features['Sub_AV'] = stars_df_features['Amag'] + stars_df_features['Vmag']

stars_df_features.head()

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,Amag_SQ,Vmag_SQ,B-V_SQ,Sum_AV,Sub_AV
0,9.10,3.54,1.39,0.482,16.845016,3,283.754574,82.8100,0.232324,25.945016,25.945016
1,9.27,21.90,3.10,0.999,20.972221,5,439.834036,85.9329,0.998001,30.242221,30.242221
2,6.61,2.81,0.63,-0.019,13.853532,1,191.920338,43.6921,0.000361,20.463532,20.463532
3,8.06,7.75,0.97,0.370,17.506509,3,306.477840,64.9636,0.136900,25.566509,25.566509
4,8.55,2.87,1.11,0.902,15.839409,4,250.886893,73.1025,0.813604,24.389409,24.389409


In [4]:
X = stars_df_features.drop('TargetClass', axis=1, inplace=False)
Y = stars_df_features['TargetClass']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state=21, stratify=Y)

In [5]:
X_scaler = skl.preprocessing.StandardScaler()

In [6]:
X_scaler.fit(X_train)

StandardScaler()

In [7]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=10))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [9]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [ ]:
tuner.search(X_train_scaled,Y_train,epochs=20,validation_data=(X_test_scaled,Y_test))

Trial 66 Complete [00h 00m 26s]
val_accuracy: 0.057851240038871765

Best val_accuracy So Far: 0.057851240038871765
Total elapsed time: 00h 03m 11s

Search: Running Trial #67

Value             |Best Value So Far |Hyperparameter
tanh              |sigmoid           |activation
3                 |7                 |first_units
6                 |3                 |num_layers
7                 |7                 |units_0
7                 |9                 |units_1
7                 |9                 |units_2
1                 |None              |units_3
1                 |None              |units_4
1                 |None              |units_5
20                |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
0                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/20
187/242 [======================>.......] - ETA: 0s - loss: -0.8324 - accuracy: 0.0577